In [1]:
## Here we will write out the matrix elements for our non-eq setup

from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

In [3]:
NL1 = 2
NL2 = 2
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

In [4]:
## Define the Hamiltonian 

w0list = np.linspace(1,1,N)
glist = np.linspace(0.0016,0.0016,N-1)

delta = 1

H_S = create_hamiltonian(w0list,glist,delta,N)

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]  # Create list of sigma_- operators

eigenergies,eigstates=H_S.eigenstates()

In [5]:
## Let us compute the thermal density matrix
beta1 = 1  #left baths
beta2 = 0.7 #right baths

rho_th = (-beta1*H_S).expm()/((-beta1*H_S).expm()).tr() #left thermal density matrix
print(rho_th)

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[3.80777093e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.03175783e-03 3.29634248e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.29634248e-06 1.02846676e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.61523826e-02
  1.79974202e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.79974202e-04
  5.63320690e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.53616443e-01]]


In [6]:
## Now we set parameters for the  non-eq setup

beta_list_left = np.linspace(beta1,beta1,NL1)
beta_list_right = np.linspace(beta2,beta2,NL2)

mu_list_left = np.linspace(-0.5,-0.5,NL1)
mu_list_right = np.linspace(-0.5,-0.5,NL2)

gamma_list_left = np.linspace(1,1,NL1)  #coupling strengths to left baths
gamma_list_right = np.linspace(1,1,NL2)  #coupling strengths to right baths

tb = 0.01

In [8]:
number = len(eigenergies)

## Now we will write out the matrix elements

A = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        sum = 0
        vi = eigstates[i]
        vk = eigstates[k]
        proj_i = vi*vi.dag()
        proj_k = vk*vk.dag()
        for y in range(number):
            for l in range(NL1):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*C(k,y,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*D(y,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

            for l in range(NL2):
                proj_y = eigstates[y]*eigstates[y].dag()
                op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*C(k,y,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op1 + op1.dag())*vi

                op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*D(y,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += vi.dag()*(op2 + op2.dag())*vi

        A[i,k] = sum


In [9]:
print(number)  #should be 2^N

64


In [10]:
rho_th_new = np.zeros((number,number),dtype=complex)

for i in range(number):
    for k in range(number):
        vi = eigstates[i]
        vk = eigstates[k]

        rho_th_new[i,k] = vi.dag()*rho_th*vk

#print(rho_th_new)

rho_th_diag = [rho_th_new[i,i] for i in range(number)]
print(rho_th_diag)


[np.complex128(0.1536164432357538+0j), np.complex128(0.05651233129231378+0j), np.complex128(0.056463896272977196+0j), np.complex128(0.05633178086692867+0j), np.complex128(0.05615180727947186+0j), np.complex128(0.055972408687012254+0j), np.complex128(0.055841443273902114+0j), np.complex128(0.020789724855111807+0j), np.complex128(0.020771906607265127+0j), np.complex128(0.02075165768501958+0j), np.complex128(0.020723304065517863+0j), np.complex128(0.020710947145894693+0j), np.complex128(0.020674601227246826+0j), np.complex128(0.02065709548273863+0j), np.complex128(0.020629746935722004+0j), np.complex128(0.020591098428797645+0j), np.complex128(0.020560107729941855+0j), np.complex128(0.020542918945809906+0j), np.complex128(0.020509188788367952+0j), np.complex128(0.020490634791516974+0j), np.complex128(0.02043729694990147+0j), np.complex128(0.020361504768194885+0j), np.complex128(0.007648112361806576+0j), np.complex128(0.007641557394746085+0j), np.complex128(0.007634108232546068+0j), np.comp

In [11]:
## Now we use the lin algebra solver to solve the equation Ax = b, where b is all zeroes

b = np.zeros((number),dtype=complex)

x = np.linalg.solve(A,b)

print(x)

[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j
 -0.-0.j]


In [12]:
print(A)

[[ 1.92425554e+000+0.00000000e+000j -9.80919814e-001+0.00000000e+000j
  -1.40642080e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-3.20886592e-001-2.32425841e-018j  2.58464343e+000-1.37032990e-017j
  -7.66662026e-018-1.56452070e-017j ... -1.62714285e-034+2.27222091e-051j
  -7.37598925e-035-4.38532762e-050j -1.11210758e-065-1.34086745e-081j]
 [-4.59790050e-001+3.44911816e-018j -2.28905994e-017+4.35036122e-017j
   2.87124211e+000+8.39366538e-017j ... -1.00009272e-034+6.40869241e-051j
  -4.37843403e-035-3.14414905e-051j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -2.58743655e-036-3.40883825e-053j
  -7.33319434e-036-4.64053096e-052j ...  5.05648233e+000+3.75676053e-017j
  -5.44872851e-019-3.12954818e-017j -1.21880827e+000+1.17093835e-017j]
 [ 3.48028785e-098+7.47416528e-114j -1.17253583e-036-5.58733678e-053j
  -3.26881888e-036+1.44290672e-052j ...  4.50348276e-017-5.084569

In [13]:
## Now we can add one more constraint, that the sum of all the elements of x should be 1, which is basically another row in the matrix A consisting of all 1

#A = np.vstack([A,np.ones((1,number))])

#print(A)

print(np.linalg.matrix_rank(A))

q,r = np.linalg.qr(A.T)

print(r)
diag_r = [r[i][i] for i in range(number)]

print(np.linalg.matrix_rank(A))





63
[[-3.16095290e+00+0.00000000e+00j  9.97419404e-01-6.20543172e-18j
   1.55741902e+00+4.24506949e-17j ... -4.00256467e-34-1.83106206e-51j
  -1.81476300e-34+5.55579689e-51j -7.14272386e-66+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -3.29069855e+00+0.00000000e+00j
   1.04537166e-01-7.46298954e-17j ... -6.20556067e-32-1.89224223e-49j
  -4.78933860e-32+7.82713468e-49j -1.58296096e-34+1.11743076e-50j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  -3.19938913e+00+0.00000000e+00j ... -5.69784087e-32-1.97774866e-48j
  -3.43254702e-32-3.96006799e-48j -1.04807216e-34+1.63582324e-50j]
 ...
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ... -4.36195841e+00+0.00000000e+00j
   1.47391898e+00-1.75437989e-16j  2.88803943e+00-2.12125734e-16j]
 [ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
  -3.34974752e+00+0.00000000e+00j  3.3497475

In [14]:
print(diag_r)

[np.complex128(-3.1609528981437647+0j), np.complex128(-3.2906985472354218+0j), np.complex128(-3.199389126809216+0j), np.complex128(-2.997704245146391+0j), np.complex128(-2.916227856784598+0j), np.complex128(-2.9200694365975406+0j), np.complex128(-3.130685170559283+0j), np.complex128(-3.402151302602078+0j), np.complex128(-3.4118118269959585+0j), np.complex128(-3.05536716594929+0j), np.complex128(-3.344886968055592+0j), np.complex128(-3.22796869664965+0j), np.complex128(-3.1274803600725085+0j), np.complex128(-3.326475949491587+0j), np.complex128(-3.2447031784906457+0j), np.complex128(-3.3361071486980185+0j), np.complex128(-3.236709846750192+0j), np.complex128(-3.383348510562966+0j), np.complex128(-3.134056334094532+0j), np.complex128(-3.07596820344647+0j), np.complex128(-3.1353636319593963+0j), np.complex128(-3.1702247990612253+0j), np.complex128(-3.4887031467096152+0j), np.complex128(-3.7772484150534553+0j), np.complex128(-3.7364453924377434+0j), np.complex128(-3.691683969843675+0j), np

In [15]:
## Since last element of r is 0, we can remove the last row of  A and add a row of 1's

A_new = A[:-1]
A_new = np.vstack([A_new,np.ones((1,number))])

print(np.linalg.matrix_rank(A_new))
print(A_new.shape)
print(A_new)

64
(64, 64)
[[ 1.92425554e+000+0.00000000e+000j -9.80919814e-001+0.00000000e+000j
  -1.40642080e+000+0.00000000e+000j ...  0.00000000e+000+0.00000000e+000j
  -2.31309218e-125+0.00000000e+000j  0.00000000e+000+0.00000000e+000j]
 [-3.20886592e-001-2.32425841e-018j  2.58464343e+000-1.37032990e-017j
  -7.66662026e-018-1.56452070e-017j ... -1.62714285e-034+2.27222091e-051j
  -7.37598925e-035-4.38532762e-050j -1.11210758e-065-1.34086745e-081j]
 [-4.59790050e-001+3.44911816e-018j -2.28905994e-017+4.35036122e-017j
   2.87124211e+000+8.39366538e-017j ... -1.00009272e-034+6.40869241e-051j
  -4.37843403e-035-3.14414905e-051j  3.81455601e-067+7.72195917e-082j]
 ...
 [ 0.00000000e+000+0.00000000e+000j -2.58743655e-036-3.40883825e-053j
  -7.33319434e-036-4.64053096e-052j ...  5.05648233e+000+3.75676053e-017j
  -5.44872851e-019-3.12954818e-017j -1.21880827e+000+1.17093835e-017j]
 [ 3.48028785e-098+7.47416528e-114j -1.17253583e-036-5.58733678e-053j
  -3.26881888e-036+1.44290672e-052j ...  4.50348276e-

In [16]:
print(A.shape)

(64, 64)


In [17]:
print(np.linalg.matrix_rank(A))
print(A.shape)

63
(64, 64)


In [18]:
b[-1] = 1  ## Last element of b is 1

x = np.linalg.solve(A_new,b)
print(x)

[0.18382985-7.59757377e-20j 0.06013595-1.40442203e-18j
 0.06009805-1.74088975e-18j 0.05999464+2.11462833e-19j
 0.05985367-8.81342179e-20j 0.05971303+2.80611865e-18j
 0.05961029+2.66857747e-19j 0.01967217-5.55898453e-19j
 0.01965978+2.29144263e-19j 0.01964568+6.67300509e-19j
 0.01962595+6.09611112e-19j 0.01961734+9.71303768e-20j
 0.01959203-1.10259557e-19j 0.01957983+5.71271319e-21j
 0.01956077+1.24950726e-20j 0.01953382-6.38141441e-20j
 0.0195122 +5.10660606e-19j 0.01950021+7.23039855e-20j
 0.01947667-4.35344770e-20j 0.01946371+1.13387720e-18j
 0.01942645-1.71971030e-19j 0.01937347+1.51261030e-19j
 0.00643533-1.55438386e-19j 0.00643127-1.15724406e-19j
 0.00642666+2.01357350e-19j 0.00642126-3.46726756e-19j
 0.0064202 +4.15240048e-19j 0.00641739-3.11675128e-20j
 0.00640911-1.05008974e-19j 0.00640512+9.98616620e-21j
 0.00639888-1.33049485e-19j 0.00639439+1.45125058e-19j
 0.00639007+1.79349933e-19j 0.006383  -9.22860227e-20j
 0.00637907+5.12223873e-19j 0.00637505-1.26817329e-20j
 0.0063713

In [19]:
##Let us check if this is correct

print(np.dot(A_new,x))
print(np.dot(A[-1],x))

[ 0.00000000e+00+3.85185989e-34j -1.38777878e-17+0.00000000e+00j
  2.08166817e-17-7.70371978e-34j -6.38069098e-35-9.62964972e-35j
 -1.38777878e-17+3.85185989e-34j  2.07872712e-34-7.70371978e-34j
  6.93889390e-18+5.77778983e-34j -5.16947596e-16-1.92592994e-33j
  6.93889390e-17+5.20001085e-33j -1.38777878e-16-6.93334780e-33j
  2.77555756e-17+1.15555797e-33j  1.73472348e-17+5.53704859e-33j
 -1.52550146e-35+2.26296768e-33j  1.04083409e-16-9.62964972e-34j
  2.08166817e-17-9.14816724e-34j  3.81639165e-17-1.01111322e-33j
  4.16333634e-17-3.85185989e-34j  1.38777878e-17+1.25185446e-33j
  1.38777878e-17-4.09260113e-34j  1.73472348e-17-2.31111593e-33j
  2.77555756e-17+1.92592994e-34j  2.77555756e-17-9.62964972e-35j
  1.11022302e-16-9.02779661e-34j  4.85722573e-17-1.05926147e-33j
 -6.93889390e-18+0.00000000e+00j -1.73472348e-17+0.00000000e+00j
  1.73472348e-18+0.00000000e+00j -1.73472348e-18+2.40741243e-35j
 -5.20417043e-18-6.74075481e-34j  2.25514052e-17+2.40741243e-34j
  1.04083409e-17+4.814824

In [20]:
#We have to tranfer the basis of the solution rho matrix to computational basis from the eigenbasis

x_real = [np.real(x[i]) for i in range(number)]

rho = np.diag(x_real)

#set U matrix whose columns are the eigenvectors of the Hamiltonian

U = np.zeros((number,number),dtype=complex)
for i in range(number):
    U[:,i] = eigstates[i].full().flatten()

check = 1

print(U[:,check])
print(eigstates[check])

[ 0.00000000e+00+0.j -5.75754671e-84-0.j -7.20269094e-81-0.j
  9.97707536e-26+0.j  0.00000000e+00+0.j -9.97750790e-26+0.j
 -4.68927578e-18+0.j  7.44164867e-30+0.j  6.93446618e-39+0.j
  4.68927558e-18+0.j  2.00350271e-22+0.j  1.62645715e-28+0.j
 -2.76943731e-22+0.j  1.69750167e-16+0.j -1.69218316e-16+0.j
 -3.47076771e-19+0.j  2.16692542e-36+0.j  4.68887508e-18+0.j
  3.61763387e-22+0.j -1.69750168e-16+0.j -2.45816511e-22+0.j
  1.45324297e-17+0.j  7.48129298e-17+0.j  1.36203428e-17+0.j
  4.68870187e-18+0.j -1.46662039e-16+0.j  3.47937418e-17+0.j
  3.84671301e-17+0.j -1.38911464e-16+0.j -1.73142083e-16+0.j
 -1.15759670e-16+0.j -4.08248290e-01+0.j  1.00408924e-33+0.j
 -3.30354054e-22+0.j -4.68835463e-18+0.j  1.40153459e-16+0.j
 -4.68902623e-18+0.j  5.20265203e-17+0.j  2.06555726e-17+0.j
 -1.76440484e-16+0.j -1.19785237e-22+0.j -4.79489729e-17+0.j
  6.25997904e-17+0.j  1.98793246e-16+0.j  2.21017512e-16+0.j
  2.22708655e-16+0.j -2.92101574e-16+0.j -4.08248290e-01+0.j
  2.00903823e-22+0.j -9.

In [21]:
rho_comp = np.dot(np.conjugate(U.T),np.dot(rho,U))

print(rho_comp)
print(np.trace(rho_comp))

[[ 2.25281359e-04+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  2.10389767e-03+0.j -1.62077103e-03+0.j ...
   1.37507935e-35+0.j  6.86042256e-36+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.62077103e-03+0.j  2.72615299e-03+0.j ...
   2.41074683e-35+0.j  1.20474749e-35+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  1.37507935e-35+0.j  2.41074683e-35+0.j ...
   3.87067060e-02+0.j -1.83553046e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  6.86042256e-36+0.j  1.20474749e-35+0.j ...
  -1.83553046e-02+0.j  3.94828598e-02+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  1.83829849e-01+0.j]]
(1.0000000000000007+0j)


In [22]:
def L2_red(rho,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*C(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies) + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*D(i,k,tb,beta_list_left[l],mu_list_left[l],gamma_list_left[l],eigenergies)
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*C(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies) + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*D(i,k,tb,beta_list_right[l],mu_list_right[l],gamma_list_right[l],eigenergies)
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

In [23]:
L2_redfield = L2_red(rho_comp,eigenergies,eigstates,beta_list_left,beta_list_right,mu_list_left,mu_list_right,gamma_list_left,gamma_list_right,tb)

In [24]:
print(L2_redfield)

[[-5.61165272e-03+0.00000000e+00j  5.27246493e-04-4.36789105e-06j
   8.27141338e-04+3.93610492e-06j ...  1.07897453e-25-1.68336131e-26j
   1.85851083e-25-1.03727623e-25j  0.00000000e+00+0.00000000e+00j]
 [ 5.27246493e-04+4.36789105e-06j  4.43269918e-03+0.00000000e+00j
  -7.61083744e-03+1.78205527e-06j ... -5.82076140e-21-6.76515154e-21j
  -1.28152420e-21-1.46845455e-21j -3.38730619e-30-1.89473797e-30j]
 [ 8.27141338e-04-3.93610492e-06j -7.61083744e-03-1.78205527e-06j
   7.47764474e-03+0.00000000e+00j ...  4.82619514e-21+6.27883308e-21j
   1.05463056e-21+1.37595912e-21j -1.51294401e-28-4.81735389e-30j]
 ...
 [ 1.07897453e-25+1.68336131e-26j -5.82076140e-21+6.76515154e-21j
   4.82619514e-21-6.27883308e-21j ... -1.07987241e-01+0.00000000e+00j
  -3.40522996e-02-2.54055163e-05j -2.16196251e-03+8.98860026e-06j]
 [ 1.85851083e-25+1.03727623e-25j -1.28152420e-21+1.46845455e-21j
   1.05463056e-21-1.37595912e-21j ... -3.40522996e-02+2.54055163e-05j
  -1.19686596e-01+0.00000000e+00j  3.41177661e-

In [25]:
## As we can see, all the checks are completely satisfied. Thus we can arrange the values of x (real values) in a diagonal density matrix and save it in a .mat file
#We will also pass in L2_redfield operator

rho_th_check = np.zeros((number,number),dtype=complex)
for i in range(number):
    for k in range(number):
        rho_th_check[i,k] = rho_th[i,k]

print(rho_th_check)

data_dict = {"dm_ness":rho_comp, "dm_th":rho_th_new,"L2_red":L2_redfield, "dm_th_check":rho_th_check}

scipy.io.savemat(F'../matlab/ness_data_NL1 = {NL1}_NL2 = {NL2}_NM = {NM}_4.mat',data_dict)


[[3.80777093e-04+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 1.03175783e-03+0.j 3.29634248e-06+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 3.29634248e-06+0.j 1.02846676e-03+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 ...
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  5.61523826e-02+0.j 1.79974202e-04+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  1.79974202e-04+0.j 5.63320690e-02+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j ...
  0.00000000e+00+0.j 0.00000000e+00+0.j 1.53616443e-01+0.j]]
